In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/MLGroupProject/Dataset/")

In [ ]:
import numpy as np 
import pandas as pd 
import torch
from tqdm import tqdm
import pickle
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
## Load embeddings 
 
filehandler = open("Embeddings/emb_player_vec_dict", 'rb')
final_player_dict = pickle.load(filehandler)
filehandler.close()

filehandler = open("Embeddings/emb_player_stoi_dict", 'rb')
final_player_stoi = pickle.load(filehandler)
filehandler.close()

filehandler = open("Embeddings/emb_matrix", 'rb')
embed_matrix = pickle.load(filehandler)
filehandler.close()

Read the player-wise scraped data 

In [ ]:
total_players = []
for file in os.listdir("/content/drive/My Drive/MLGroupProject/Dataset/ipl_stats/"):
    if file[-1:]!='v':
        continue

    df = pd.read_csv("/content/drive/My Drive/MLGroupProject/Dataset/ipl_stats/"+file)
    print(df.isna().sum())

    total_players.extend(df['name'].to_list())

total_players = list(set(total_players))

pos          0
name         0
team_no      0
team_name    0
pts          0
mat          0
wkts         0
dots         0
4s           0
6s           0
catches      0
stumpings    0
dtype: int64
pos          0
name         0
team_no      0
team_name    0
pts          0
mat          0
wkts         0
dots         0
4s           0
6s           0
catches      0
stumpings    0
dtype: int64
pos          0
name         0
team_no      0
team_name    0
pts          0
mat          0
wkts         0
dots         0
4s           0
6s           0
catches      0
stumpings    0
dtype: int64
pos          0
name         0
team_no      0
team_name    0
pts          0
mat          0
wkts         0
dots         0
4s           0
6s           0
catches      0
stumpings    0
dtype: int64
pos          0
name         0
team_no      0
team_name    0
pts          0
mat          0
wkts         0
dots         0
4s           0
6s           0
catches      0
stumpings    0
dtype: int64
pos          0
name         0
team_

In [ ]:
len(total_players)

703

Read match data files

In [ ]:
delivery=pd.read_csv('./kaggle_data/deliveries.csv')
matches=pd.read_csv('./kaggle_data/matches.csv')  

In [ ]:
delivery.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
merged_df = pd.merge(delivery, matches, on='match_id') ## Merge the datafiles 

In [ ]:
reqd_col = ['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'total_runs', 'season']

In [ ]:
bb_df = merged_df[reqd_col] ## Select required features 

Create a list of players in IPL

In [ ]:
total_players_deliveries = bb_df['bowler'].to_list()
total_players_deliveries.extend(bb_df['batsman'].to_list())
total_players_deliveries.extend(bb_df['non_striker'].to_list())
total_players_deliveries = list(set(total_players_deliveries))

total_players_embed = list(final_player_dict.keys())

In [ ]:
different_name = [] ## store the player names which are different in the two datasets 
for player in total_players_deliveries:
    if player not in total_players_embed:
        different_name.append(player)

In [ ]:
del_to_emb = {} ## key(deliver_df_name) -> key(embed_player_name)
not_found = {}

for player in total_players_deliveries: ## Loop over all the players 
    closest_match = []
    # print(player)

    # try:
    player_fn = player.split()[0] ## Store the name of the player for matching
    player_ln = player.split()[-1]
    # except:
    #     player_fn = player.split("-")[0]
    #     player_ln = player.split("-")[-1]

    for embed_name in total_players_embed:
        # print(embed_name)

        # try:
        embed_fn = embed_name.split()[0]
        embed_ln = embed_name.split()[-1]
        # except:
        #     embed_fn = embed_name.split("-")[0]
        #     embed_ln = embed_name.split("-")[-1]

        if player_ln.lower() == embed_ln.lower(): ## Check last name
            # closest_match.append(embed_name)
            if embed_fn[0].lower() == player_fn[0].lower(): ## Find the closes match in the data 
                closest_match.append(embed_name) ## name stored as intials 
    
    closest_match = list(set(closest_match))
    if len(closest_match) == 1:
        del_to_emb[player] = closest_match[0]
        print(player, " --> ", closest_match[0])

    elif len(closest_match) == 0:
        not_found[player] = closest_match

    else:
        print(player)
        print(closest_match)
        
        final_name = input()

        if final_name != "n":
            del_to_emb[player] = closest_match[int(final_name)]

            print(player, " --> ", closest_match[int(final_name)])
        else:
            not_found[player] = closest_match


SM Katich  -->  Simon Katich
DJG Sammy  -->  Darren Sammy
Sachin Baby  -->  Sachin Baby
NM Coulter-Nile  -->  Nathan Coulter-Nile
SE Marsh  -->  Shaun Marsh
P Negi  -->  Pawan Negi
YK Pathan  -->  Yusuf Pathan
JR Hopes  -->  James Hopes
CJ Anderson  -->  Corey Anderson
S Singh
['Swapnil Singh', 'Sunny Singh', 'Simran Singh']
n
BJ Haddin  -->  Brad Haddin
Mustafizur Rahman
['Mujeeb Ur Rahman', 'Mustafizur Rahman']
1
Mustafizur Rahman  -->  Mustafizur Rahman
SB Joshi  -->  Sunil Joshi
ER Dwivedi  -->  Eklavya Dwivedi
SB Styris  -->  Scott Styris
DJ Harris  -->  Daniel Harris
K Paul  -->  Keemo Paul
A Choudhary  -->  Aniket Choudhary
Bipul Sharma  -->  Bipul Sharma
RS Gavaskar  -->  RS Gavaskar
SV Samson  -->  Sanju Samson
SS Shaikh  -->  SS Shaikh
S Lamichhane  -->  Sandeep Lamichhane
SD Lad  -->  Siddhesh Lad
NJ Maddinson  -->  Nic Maddinson
TP Sudhindra  -->  TP Sudhindra
SA Abbott  -->  Sean Abbott
IC Pandey  -->  Ishwar Pandey
AB Dinda  -->  Ashok Dinda
NB Singh  -->  Nathu Singh
PJ 

Save the emeddings 

In [ ]:
filehandler = open("/Embeddings/del_to_emb_final_old", 'wb')
pickle.dump(del_to_emb, filehandler)
filehandler.close()

filehandler = open("/Embeddings/not_found", 'wb')
pickle.dump(not_found, filehandler)
filehandler.close()

Manually assigning vectors 

In [ ]:
del_to_emb["S Singh"] = "Simran Singh"
not_found.pop("S Singh", None)

['Swapnil Singh', 'Sunny Singh', 'Simran Singh']

In [ ]:
del_to_emb["Arshdeep Singh"] = "Simran Singh"
not_found.pop("A Singh", None)

['Amit Singh', 'Anureet Singh', 'Arshdeep Singh', 'Anirudh Singh']

Find the closest name matches for the remaining players (Initials to store name)

In [ ]:
## Loop for players were not found earlier 
## key(deliver_df_name) -> key(embed_player_name)

for player in not_found.keys(): ## Loop over the players which were not found earlier 
    closest_match = []
    # print(player)

    # try:
    player_fn = player.split()[0] ## Split the name of the player
    player_ln = player.split()[-1]
    # except:
    #     player_fn = player.split("-")[0]
    #     player_ln = player.split("-")[-1]

    for embed_name in total_players_embed:
        # print(embed_name)

        # try:
        embed_fn = embed_name.split()[0]
        embed_ln = embed_name.split()[-1]
        # except:
        #     embed_fn = embed_name.split("-")[0]
        #     embed_ln = embed_name.split("-")[-1]

        if player_ln.lower() == embed_ln.lower(): ## Check last name
            closest_match.append(embed_name)
    
    closest_match = list(set(closest_match))

    print(player)
    print(closest_match)
    if len(closest_match) == 1: ## Assign the name manually by entering the index 
        inp = input()
        if inp == 'y':
            del_to_emb[player] = closest_match[0]
            print(player, " --> ", closest_match[0])

    elif len(closest_match) == 0:
        not_found[player] = closest_match

    else:
        print(player)
        print(closest_match)
        
        final_name = input()

        if final_name != "n":
            del_to_emb[player] = closest_match[int(final_name)]

            print(player, " --> ", closest_match[int(final_name)])
        # else:
        #     not_found[player] = closest_match


GH Vihari
['Hanuma Vihari']
y
GH Vihari  -->  Hanuma Vihari
CRD Fernando
['Dilhara Fernando']
y
CRD Fernando  -->  Dilhara Fernando
J Syed Mohammad
['Syed Mohammad']
y
J Syed Mohammad  -->  Syed Mohammad
BMAJ Mendis
['Jeevan Mendis', 'Ajantha Mendis']
BMAJ Mendis
['Jeevan Mendis', 'Ajantha Mendis']
0
BMAJ Mendis  -->  Jeevan Mendis
DPMD Jayawardene
['Mahela Jayawardene']
y
DPMD Jayawardene  -->  Mahela Jayawardene
S Aravind
[]
VY Mahesh
['Yo Mahesh']
y
VY Mahesh  -->  Yo Mahesh
DNT Zoysa
['Nuwan Zoysa']
y
DNT Zoysa  -->  Nuwan Zoysa
GB Hogg
['Brad Hogg']
y
GB Hogg  -->  Brad Hogg
MN van Wyk
[]
AD Mascarenhas
['Dimitri Mascarenhas']
y
AD Mascarenhas  -->  Dimitri Mascarenhas
AN Ahmed
['Shoaib Ahmed', 'Shahbaz Ahmed', 'Khaleel Ahmed']
AN Ahmed
['Shoaib Ahmed', 'Shahbaz Ahmed', 'Khaleel Ahmed']
n
KD Karthik
['Dinesh Karthik', 'Arun Karthik']
KD Karthik
['Dinesh Karthik', 'Arun Karthik']
0
KD Karthik  -->  Dinesh Karthik
JA Morkel
['Albie Morkel', 'Morne Morkel']
JA Morkel
['Albie Morkel',

In [ ]:
## Interchange del_to_emb['RG Sharma'] and del_to_emb['R Sharma']
del_to_emb['R Sharma'] = 'Rahul Sharma'
del_to_emb['RG Sharma'] = 'Rohit Sharma'

Save the final embedding 

In [ ]:
filehandler = open("/Embeddings/del_to_emb_final", 'wb')
pickle.dump(del_to_emb, filehandler)
filehandler.close()

In [ ]:
imp_dict = del_to_emb

In [ ]:
filehandler = open("del_to_emb_final", 'wb')
pickle.dump(imp_dict, filehandler)
filehandler.close()

# Create csv


In [ ]:
different_name = []
for player in total_players_deliveries:
    if player not in del_to_emb.keys():
        different_name.append(player)
different_name

['S Aravind',
 'MN van Wyk',
 'AN Ahmed',
 'S Vidyut',
 'FH Edwards',
 'DJ Muthuswami',
 'MB Parmar',
 'Harmeet Singh (2)',
 'SS Sarkar',
 'Gagandeep Singh',
 'A Singh',
 'RW Price']

Function to create the ball by ball csv by replacing batsman, non-striker and bowler name by their respective vectors 

In [ ]:
def create_bbb_df(df, final_player_dict, different_name):

    col = ['inning', 'over', 'ball', 'season', 'batting_team', 'bowling_team']
    col.extend(['batsman_name_' + str(i) for i in range(8)])
    col.extend(['non_striker_name_' + str(i) for i in range(8)])
    col.extend(['bowler_name_' + str(i) for i in range(8)])
    col.extend(['total_runs'])

    df_new = pd.DataFrame(columns=col)
    for idx in range(len(df)):
        batsman_name = " ".join(df.iloc[idx]['batsman'].split())
        non_striker_name = " ".join(df.iloc[idx]['non_striker'].split())
        bowler_name = " ".join(df.iloc[idx]['bowler'].split())

        if (batsman_name in different_name) or (non_striker_name in different_name) or (bowler_name in different_name) :
            continue

        new_element = [float(df.iloc[idx]['inning']), float(df.iloc[idx]['over']), 
                       float(df.iloc[idx]['ball']), float(df.iloc[idx]['season']) - 2008,
                        team_dict[df.iloc[idx]['batting_team']], team_dict[df.iloc[idx]['bowling_team']]]

        new_element.extend(final_player_dict[del_to_emb[batsman_name]][0,:].tolist()) 
        new_element.extend(final_player_dict[del_to_emb[non_striker_name]][0,:].tolist()) 
        new_element.extend(final_player_dict[del_to_emb[bowler_name]][0,:].tolist())                        

        new_element.extend([float(df.iloc[idx]['total_runs'])])
        # print(new_element)
        df_new.loc[len(df_new.index)] = new_element
        # break
     

    return df_new

In [ ]:
final_df = create_bbb_df(bb_df, final_player_dict, different_name)

In [ ]:
# final_df.to_csv("Ball_by_ball.csv")

In [ ]:
final_df.shape, bb_df.shape

((176235, 31), (179078, 11))